# LSTM

Test of simple MLP models with different amount of neurons, hidden layers and size of input vector

## Load Data

In [149]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
 
import pandas as pd
import numpy as np

from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [150]:
df = pd.read_csv("data_N_3000_noise_level_0.05.csv")
df_diff = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [151]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        all_data = self.__make_stack(dataframe)
        self.X_ = all_data[:, :-n_dot_parameters]
        self.y_ = all_data[:, -n_dot_parameters:]
        
    def __len__(self):
        return len(self.X_)
    
    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]
    
    def __make_stack(self, df: pd.DataFrame):
        stacks = [df.iloc[:-self.n_dots]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values for i in range(1, self.n_dots + 1)]
        return torch.tensor(np.hstack(stacks), dtype=torch.float32)
    
    def plotData(self, i = 0, f = -1):
        fig = px.line_3d(x = self.X_[i:f, 1], y = self.X_[i:f, 2], z = self.X_[i:f, 3])
        fig.show()

In [152]:
N_DOTS = 100
N_DOT_PARAMETERS = 4
N_LSTM_LAYERS = 10

In [153]:
dataset = SequentDataset(df_diff[0:2500], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df_diff[2500:3000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
dataset.plotData()

In [154]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

## Create Model

In [155]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_layer_size=100, output_size=4, num_layers=10):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)
        
        
        
        # Define the output layer
        self.linear = nn.Linear(hidden_layer_size, output_size)
    
    def forward(self, x):

        batch_size = x.size(0)
        x = x.to(device)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size).requires_grad_()
        h0 = h0.to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size).requires_grad_()
        c0 = c0.to(device)

        out, _ = self.lstm(x)
        
        # print(out.size())
        
        # Pass through fully connected layer
        out = self.linear(out)  # We want the output corresponding to the last time step
        return out


In [156]:
model = LSTMModel(input_size=N_DOTS * N_DOT_PARAMETERS, num_layers=N_LSTM_LAYERS).to(device)
# loss_model = nn.MSELoss(reduction='mean')
loss_model = nn.L1Loss(reduction='mean')
opt = Adam(model.parameters(), lr=0.001)

In [157]:
EPOCH = 250
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    train_loss = []
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)
        
        y_pred = model(X)
        loss = loss_model(y_pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)
    
    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

Epoch [1/250], train_loss = 0.0878, val_loss = 0.0490
Epoch [2/250], train_loss = 0.0458, val_loss = 0.0450
Epoch [3/250], train_loss = 0.0421, val_loss = 0.0420
Epoch [4/250], train_loss = 0.0398, val_loss = 0.0388
Epoch [5/250], train_loss = 0.0374, val_loss = 0.0399
Epoch [6/250], train_loss = 0.0374, val_loss = 0.0368
Epoch [7/250], train_loss = 0.0366, val_loss = 0.0381
Epoch [8/250], train_loss = 0.0362, val_loss = 0.0389
Epoch [9/250], train_loss = 0.0365, val_loss = 0.0375
Epoch [10/250], train_loss = 0.0360, val_loss = 0.0371
Epoch [11/250], train_loss = 0.0362, val_loss = 0.0371
Epoch [12/250], train_loss = 0.0364, val_loss = 0.0369
Epoch [13/250], train_loss = 0.0363, val_loss = 0.0389
Epoch [14/250], train_loss = 0.0366, val_loss = 0.0365
Epoch [15/250], train_loss = 0.0360, val_loss = 0.0368
Epoch [16/250], train_loss = 0.0361, val_loss = 0.0375
Epoch [17/250], train_loss = 0.0363, val_loss = 0.0379
Epoch [18/250], train_loss = 0.0362, val_loss = 0.0370
Epoch [19/250], tra

In [158]:
X.size()

torch.Size([32, 400])

In [159]:
start, _  = dataset[0]
start = torch.reshape(start, (1, start.size(0)))
predictions = []
model.eval()

for i in range(1, N_LSTM_LAYERS):
    model(start)
    start, _ = dataset[i]
    start = torch.reshape(start, (1, start.size(0)))

for i in range(N_LSTM_LAYERS ,500):
    with torch.no_grad():
        pred = model(start)
        predictions.append(pred)
        start = torch.hstack([start, pred])[:, N_DOT_PARAMETERS:]
        # start = pred   

In [160]:
start

tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04,  9.9027e-01,
         -7.1853e-03,  6.5518e-03,  1.6971e-04,  9.9027e-01, -7.1853e-03,
          6.5518e-03,  1.6971e-04,  9.9027e-01, -7.1853e-03,  6.5518e-03,
          1.6971e-04,  9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04,
          9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04,  9.9027e-01,
         -7.1853e-03,  6.5518e-03,  1.6971e-04,  9.9027e-01, -7.1853e-03,
          6.5518e-03,  1.6971e-04,  9.9027e-01, -7.1853e-03,  6.5518e-03,
          1.6971e-04,  9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04,
          9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04,  9.9027e-01,
         -7.1853e-03,  6.5518e-03,  1.6971e-04,  9.9027e-01, -7.1853e-03,
          6.5518e-03,  1.6971e-04,  9.9027e-01, -7.1853e-03,  6.5518e-03,
          1.6971e-04,  9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04,
          9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04,  9.9027e-01,
         -7.1853e-03,  6.5518e-03,  1.

In [161]:
predictions

[tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7.1853e-03,  6.5518e-03,  1.6971e-04]]),
 tensor([[ 9.9027e-01, -7

In [162]:
np.vstack(list(map(lambda x: x.detach().numpy().reshape((-1)), predictions))).T

array([[ 9.9027169e-01,  9.9027169e-01,  9.9027163e-01, ...,
         9.9027169e-01,  9.9027169e-01,  9.9027169e-01],
       [-7.1852989e-03, -7.1852989e-03, -7.1852989e-03, ...,
        -7.1852952e-03, -7.1852952e-03, -7.1852952e-03],
       [ 6.5518022e-03,  6.5518003e-03,  6.5518040e-03, ...,
         6.5518059e-03,  6.5518059e-03,  6.5518059e-03],
       [ 1.6970746e-04,  1.6971119e-04,  1.6971119e-04, ...,
         1.6971119e-04,  1.6971119e-04,  1.6971119e-04]], dtype=float32)

In [163]:
p_x = pd.DataFrame(np.vstack(list(map(lambda x: x.detach().numpy().reshape((-1)), predictions))), columns=["Time", "X", "Y", "Z"])
_, y  = dataset[:500]
p_y = pd.DataFrame(y.detach().numpy(), columns=["Time", "X", "Y", "Z"])

p_x = p_x.cumsum()
p_y = p_y.cumsum()

In [164]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [165]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()